In [1]:
%matplotlib qt5
import sys
sys.path.append('../')
from process_files import *
from func_science import *

qt.qpa.xcb: X server does not support XInput 2


qt.qpa.xcb: QXcbConnection: XCB error: 1 (BadRequest), sequence: 169, resource id: 132, major code: 130 (Unknown), minor code: 47


#### 8. Invert the Stokes profiles using VFISV for FPI based instruments

In [2]:
# Load the metadata
config = configobj.ConfigObj('../config.ini')
line = 'Fe_I_6302'
line_ = 'Fe_I_6301'
dkdir = config['darks']['directory']
scdir = config['science']['directory']
settings = [f for f in os.listdir(scdir) if 'settings' in f]
settings = scdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
# Other numbers
iline = get_line_num(settings, line, 0)
linestr = 'Line_' + str(iline)
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(scdir+os.sep+line))//3
wavelength = np.float128(settings[linestr]['LineWavelength'])-0.25
wave_range = np.float128(settings[linestr]['WaveScanRange'])
wave_step = wave_range/(np.int16(settings[linestr]['NWavePoints'])-1)
geff = np.float128(config['science'][line+'/geff'])
#
suffix_dq = '_' # data quality suffix for image reconstructed vs non-reconstructed data ('' : reconstructed, '_' : not reconstructed)
l0corrdir = config['science'][line+'/l0corrdir'+suffix_dq]
pol_files = [l0corrdir+os.sep+f for f in sorted(os.listdir(l0corrdir))]
invdir = config['science']['invdir']
tempdir = os.sep.join(l0corrdir.split(os.sep)[0:-1]).replace(line, line_)
tempdir_in = tempdir.replace(os.sep+'L0'+os.sep, os.sep+'L1'+os.sep)+os.sep+'tempdir_in'+suffix_dq
tempdir_out = tempdir.replace(os.sep+'L0'+os.sep, os.sep+'L1'+os.sep)+os.sep+'tempdir_out'+suffix_dq
if not os.path.exists(tempdir_in): os.makedirs(tempdir_in)
if not os.path.exists(tempdir_out): os.makedirs(tempdir_out)
#
ff_file = config['flats'][line+'/pol']
tp_file = config['targetplate'][line+'/l0data']
nwav = 10
config['science'][line_+'/nwav'] = nwav
config.write()

In [3]:
# Read the template spectrum from the master flat 
lines_ff = pf.open(ff_file)
align_pars = pf.open(tp_file)
affine12 = align_pars[3].data[1]
#
lines_pol1 = lines_ff[7].data
lines_pol1 = np.fliplr(lines_pol1)
lines_pol2 = lines_ff[8].data
lines_pol2 = np.flipud(np.fliplr(lines_pol2))
for w in range(nwav):
    lines_pol2[:,:,w] = affine_transform(lines_pol2[:,:,w], affine12, mode='nearest')
lines_pol = lines_pol1+lines_pol2
zeros = 0*lines_pol
lines_pol_ = np.array([lines_pol, zeros, zeros, zeros])
lines_pol_ = np.moveaxis(lines_pol_, 0, 2)
lines_pol_ = lines_pol_[:,:,:,0:nwav]
#

In [4]:
# Invert template spectral profiles
sprefix = ['ii', 'qq', 'uu', 'vv']
tfile = open(tempdir_in+os.sep+'file_list.txt', 'w')
# sc = pf.open(pfile)[1].data
ny, nx, ns, nw = lines_pol_.shape
nwav = nw
for s in np.arange(ns):
    for w in np.arange(nw):
        file_name = sprefix[s] + '_' + f'{w:02}' + '.fits'  
        hdu = pf.PrimaryHDU(lines_pol_[:,:,s,w])
        hdu.writeto(tempdir_in+os.sep+file_name, overwrite=True)
        tfile.write(file_name + '\n')
tfile.close()

# create the map file
params_file = 'pol_flat_params.dat'
stokes_file = 'pol_flat_stokes.dat'
#
tfile = open(invdir+os.sep+'hellride2.map', 'w')
tfile.write(f'xmin {1} \t \n')
tfile.write(f'xmax {nx} \t \n')
tfile.write(f'ymin {1} \t \n')
tfile.write(f'ymax {ny} \t \n')
tfile.write(f'dirin {tempdir_in+os.sep} \n')
tfile.write(f'dirout {tempdir_out+os.sep} \n')
tfile.write(f'invfileout {params_file} \n')
tfile.write(f'stokfileout {stokes_file} \n')
tfile.write(f'fitslist file_list.txt \n')
tfile.close()

# create the line file
noise = 0.001
wave_steps = np.arange(-nwav//2+1, nwav//2+1)*wave_step*1e3
#
tfile = open(invdir+os.sep+'hellride2.line', 'w')
tfile.write(f'cen_wav {wavelength:.3f} \n')
tfile.write(f'geff {geff:.3f} \n')
tfile.write(f'noise {noise:.5f} \n')
tfile.write(f'nwav {nwav} \n')
tfile.write(f'cont_pix {nwav} \n')
tfile.write(f'wavpos ')
for w in wave_steps: tfile.write(f'{w} ')
tfile.write('\n')
tfile.write('telluric ini 9 \n')
tfile.write('telluric end 10 \n')
tfile.write('instrument hellride \n')
tfile.close()

# create the filter file
et1_r, et2_r = 0.93, 0.93
et1_s = int(int(settings['Etalon_1']['ZSpacing'])*1e4)
et2_s = int(int(settings['Etalon_2']['ZSpacing'])*1e4)
tfile = open(invdir+os.sep+'hellride2.filter', 'w')
tfile.write(f'reflectivity et1 {et1_r:0.3} \n')
tfile.write(f'reflectivity et2 {et2_r:0.3} \n')
tfile.write(f'distance plates et1 {et1_s} \n')
tfile.write(f'distance plates et2 {et2_s} \n')
tfile.close()
sp.run(f'mpirun -machinefile {invdir}/hostfile -n 16 {invdir}/vfisv_fpi.x -map={invdir}/hellride2.map -line={invdir}/hellride2.line -filter={invdir}/hellride2.filter', shell=True)

[mpiexec@oig] HYDU_process_mfile_token (lib/utils/args.c:300): token slots not supported at this time
[mpiexec@oig] HYDU_parse_hostfile (lib/utils/args.c:345): unable to process token
[mpiexec@oig] mfile_fn (mpiexec/options.c:304): error parsing hostfile
[mpiexec@oig] match_arg (lib/utils/args.c:159): match handler returned error
[mpiexec@oig] HYDU_parse_array (lib/utils/args.c:181): argument matching returned error
[mpiexec@oig] parse_args (mpiexec/get_parameters.c:315): error parsing input array
[mpiexec@oig] HYD_uii_mpx_get_parameters (mpiexec/get_parameters.c:47): unable to parse user arguments
[mpiexec@oig] main (mpiexec/mpiexec.c:49): error parsing parameters


CompletedProcess(args='mpirun --hostfile /home/pruthvi/vfisv_merged/src_fpi/hostfile -n 16 /home/pruthvi/vfisv_merged/src_fpi/vfisv_fpi.x -map=/home/pruthvi/vfisv_merged/src_fpi/hellride2.map -line=/home/pruthvi/vfisv_merged/src_fpi/hellride2.line -filter=/home/pruthvi/vfisv_merged/src_fpi/hellride2.filter', returncode=255)

In [ ]:
sc = pf.open(pol_files[-1])[1].data
show_img_series(sc[:,:,0,0:nwav], fps=1)

In [ ]:
# Invert the observed stoke profiles
for pfile in pol_files:
    sprefix = ['ii', 'qq', 'uu', 'vv']
    tfile = open(tempdir_in+os.sep+'file_list.txt', 'w')
    sc = pf.open(pfile)[1].data[:,:,:,0:11]
    ny, nx, ns, nw = sc.shape
    nwav = nw
    for s in np.arange(ns):
        for w in np.arange(nw):
            file_name = sprefix[s] + '_' + f'{w:02}' + '.fits'  
            hdu = pf.PrimaryHDU(sc[:,:,s,w])
            hdu.writeto(tempdir_in+os.sep+file_name, overwrite=True)
            tfile.write(file_name + '\n')
    tfile.close()

    # create the map file
    pfile_name = os.path.split(pfile)[-1]
    params_file = pfile_name.replace('.fits', '_params.dat')
    stokes_file = pfile_name.replace('.fits', '_stokes.dat')
    #
    tfile = open(invdir+os.sep+'hellride2.map', 'w')
    tfile.write(f'xmin {1} \t \n')
    tfile.write(f'xmax {nx} \t \n')
    tfile.write(f'ymin {1} \t \n')
    tfile.write(f'ymax {ny} \t \n')
    tfile.write(f'dirin {tempdir_in+os.sep} \n')
    tfile.write(f'dirout {tempdir_out+os.sep} \n')
    tfile.write(f'invfileout {params_file} \n')
    tfile.write(f'stokfileout {stokes_file} \n')
    tfile.write(f'fitslist file_list.txt \n')
    tfile.close()

    # create the line file
    noise = 1.0/np.sqrt(sc.mean())
    wave_steps = np.arange(-nwav//2+1, nwav//2+1)*wave_step*1e3
    # print(wave_steps)
    #
    tfile = open(invdir+os.sep+'hellride2.line', 'w')
    tfile.write(f'cen_wav {wavelength:.3f} \n')
    tfile.write(f'geff {geff:.3f} \n')
    tfile.write(f'noise {noise:.5f} \n')
    tfile.write(f'nwav {nwav} \n')
    tfile.write(f'cont_pix {nwav} \n')
    tfile.write(f'wavpos ')
    for w in wave_steps: tfile.write(f'{w} ')
    tfile.write('\n')
    tfile.write('telluric ini 9 \n')
    tfile.write('telluric end 10 \n')
    tfile.write('instrument hellride \n')
    tfile.close()

    # create the filter file
    et1_r, et2_r = 0.93, 0.93
    et1_s = int(int(settings['Etalon_1']['ZSpacing'])*1e4)
    et2_s = int(int(settings['Etalon_2']['ZSpacing'])*1e4)
    tfile = open(invdir+os.sep+'hellride2.filter', 'w')
    tfile.write(f'reflectivity et1 {et1_r:0.3} \n')
    tfile.write(f'reflectivity et2 {et2_r:0.3} \n')
    tfile.write(f'distance plates et1 {et1_s} \n')
    tfile.write(f'distance plates et2 {et2_s} \n')
    tfile.close()
    sp.run(f'mpirun -n 16 {invdir}/vfisv_fpi.x -map={invdir}/hellride2.map -line={invdir}/hellride2.line -filter={invdir}/hellride2.filter', shell=True)